# Machine Learning with Spark ML

### In this notebook, we will explore machine learning using Spark ML. We will exploit Spark ML's high-level APIs built on top of DataFrames to create and tune machine learning pipelines. Spark ML Pipelines enable combining multiple algorithms into a single pipeline or workflow. We will utilize Spark ML's feature transformers to convert, modify and scale the features that will be used to develop the machine learning model. Finally, we will evaluate and cross validate our model to demonstrate the process of determining a best fit model and load the results in the database.

### We are using machine learning to try to predict records that a human has not seen or vetted before. We will use these predictions to sort the highest priority records for a human to look at. We will use as a training set for the algorithm fake data that has been vetted by an analyst as high, medium or low.¶

### We will use generated travel data that has been examined for patterns of Human Trafficking from a DB2 table to do the machine learning.  

## Table of contents

1. [Create Version](#version)
1. [Import Libraries](#install)
1. [Read from Object Storage](#object)
1. [Transform the data](#transform)
1. [Feature Engineering](#engineering)
1. [Model the data](#model)
1. [Setup the Pipeline](#pipeline)
1. [Train the model](#train)
1. [Evaluate results](#evaluate)
1. [Hyperparameter Tuning](#tuning)
1. [Score the records](#score)
1. [Insert Credentials](#credentials)
1. [Write Results](#write)
1. [Create New Version](#version2)
1. [Schedule Job](#schedule)
1. [Revert to Version](#revert)
1. [Even More Help](#help)


<a id="version"></a>
## Create Version 

Save a version of the notebook by selecting <b>File</b> > <b>Save Version</b> 
<img alt="IBM Bluemix.Get started now" src="https://raw.githubusercontent.com/jpatter/LMCO/master/Lab-1/images/FileOptions.PNG" > or by selecting the <b>Versions</b> icon and selecting <b>Save Version</b>. <img alt="IBM Bluemix.Get started now" src="https://raw.githubusercontent.com/jpatter/LMCO/master/Lab-1/images/versions-button.png" ><br>
You can have up to ten (10) versions of a notebook.   Notebook versions are saved in a FIFO manner.

## Upgrade Watson Machine Learning Client library


In [18]:
!pip install --upgrade watson-machine-learning-client-V4

  Using cached https://files.pythonhosted.org/packages/5b/85/d1ae875e6a2247bf40f1da183dca698415e6fa42fa002a4d8ef7142df0cf/watson_machine_learning_client_V4-1.0.60-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/cd/80/5bb262050dd2f30f8819626b7c92339708fe2ed7bd5554c8193b4487b367/tqdm-4.42.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e8/74/6e4f91745020f967d09332bb2b8b9b10090957334692eb88ea4afe91b77f/urllib3-1.25.8-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/b9/63/df50cac98ea0d5b006c55a399c3bf1db9da7b5a24de7890bc9cfd5dd9e99/certifi-2019.11.28-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/51/bd/23c926cd341ea6b7dd0b2a00aba99ae0f828be89d72b2190f27c11d4b7fb/requests-2.22.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/12/d1/a6502c2f5c15b50f5dd579fc1c52b47edf6f2e9f682aed917dd7565b3e60/pandas-1.0.0-cp36-cp36m-manylinux1_x86_64.whl
  Using cached htt

In [20]:
import sys,os,os.path
from watson_machine_learning_client import WatsonMachineLearningAPIClient

## Verify Spark version and existence of Spark

In [22]:
print('The spark version is {}.'.format(spark.version))
# !pip install findspark
# !pip install pyspark==2.3.3

The spark version is 2.3.4.


In [24]:
# print('The spark version is {}.'.format(spark.version))


<a id="install"></a>
##  Import the required libraries

In [26]:
#Imports for Spark
from pyspark.ml.feature import StringIndexer, IndexToString
from pyspark.ml.feature import Bucketizer
from pyspark.mllib.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.feature import Normalizer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.classification import NaiveBayes, DecisionTreeClassifier,RandomForestClassifier
from pyspark.sql.functions import year
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Imports for pixiedust
from pixiedust.display import *




<a id="object"></a>
## Read Data Asset - female_human_trafficking - See Lab Instructions

In [28]:
# from pyspark.sql import SparkSession
# sparkSession = SparkSession(spark).builder.getOrCreate()
# # @hidden_cell
# # The following code contains the credentials for a connection in your Project.
# # You might want to remove those credentials before you share your notebook.
# from project_lib import Project
# project = Project.access()
# female_human_trafficking_credentials = project.get_connected_data(name="female_human_trafficking")
# print(female_human_trafficking_credentials)
# data_df_1 = sparkSession.read.format('jdbc') \
#     .option('url', '{}{}:{}/{}'.format('jdbc:db2://', female_human_trafficking_credentials['host'], '50000', female_human_trafficking_credentials['database'])) \
#     .option('dbtable', 'TRAFFICKING.FEMALE_HUMAN_TRAFFICKING') \
#     .option('user', female_human_trafficking_credentials['username']) \
#     .option('password', female_human_trafficking_credentials['password']).load()
# data_df_1.show(5)
from pyspark.sql import SQLContext


trafficking_df = SQLContext(sc).read.csv('/project_data/data_asset/female_human_trafficking.csv', header='true', inferSchema = 'true')
trafficking_df.show(5)


+-----------+-------------+-----------+--------------------+------+-------------------+-------------+------------------+--------------------+--------------------+-----------+---------------+----------------+---------------------+-----------------+-----------------------+----------------------------+--------------------+----------------------------+----------------------+------------------------------+----------------------+------------------------------+------------------------+--------------------+---+
|INTERNAL_ID|VETTING_LEVEL|DESCRIPTION|                NAME|GENDER|         BIRTH_DATE|BIRTH_COUNTRY|BIRTH_COUNTRY_CODE|          OCCUPATION|             ADDRESS|        SSN|PASSPORT_NUMBER|PASSPORT_COUNTRY|PASSPORT_COUNTRY_CODE|COUNTRIES_VISITED|COUNTRIES_VISITED_COUNT|ARRIVAL_AIRPORT_COUNTRY_CODE|ARRIVAL_AIRPORT_IATA|ARRIVAL_AIRPORT_MUNICIPALITY|ARRIVAL_AIRPORT_REGION|DEPARTURE_AIRPORT_COUNTRY_CODE|DEPARTURE_AIRPORT_IATA|DEPARTURE_AIRPORT_MUNICIPALITY|DEPARTURE_AIRPORT_REGION|        

## Read Data Asset - Occupations - See Lab Instructions
The occupations listed in the female human trafficking file are too numerous to use as input to a machine learning model. We will categorize these occupations into 15 categories by joining with two other files. The Occupation.csv file contains a mapping of the occupations in the female human trafficking table to a category code. The Categories.csv file contains each code followed by the category name. This information needs to be joined to the female human trafficking table.  

In [30]:
# Insert Categories credentials in this cell after the comments
# make CERTAIN that if credentials gets generated as credentials_1 or credentials_2 or credentials_n where n is a number
# to rename as credentials
#Put cursor on the next line to Insert to code 
from project_lib import Project
project = Project.access()
credentials = project.get_storage_metadata()
print(credentials)

{'type': 'assetfiles', 'guid': 'cd700957-2bbc-47b3-bdb5-06c3e848081b'}


In [32]:
# credentials = {
#     'endpoint' : credentials['URL'],
#     'secret_key' : credentials['SECRET_KEY'], 
#     'access_key' : credentials['ACCESS_KEY'],
#     'container' : credentials['BUCKET']
# }

# import ibmos2spark

# cos = ibmos2spark.CloudObjectStorage(sc, credentials, auth_method='')

# from pyspark.sql import SparkSession
# spark = SparkSession.builder.getOrCreate()
# occupations = spark.read\
#   .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
#   .option('header', 'true')\
#   .load(cos.url('data_asset/Occupation*.csv', credentials['container']))
occupations = SQLContext(sc).read.csv('/project_data/data_asset/Occupation_csv_cotgztm25oru4xlskc5cgk2kb.csv', header='true', inferSchema = 'true')
occupations.show(5)
occupations.take(5)

+--------------------+----+
|          OCCUPATION|Code|
+--------------------+----+
|         Chiropodist|   6|
|Engineer, structural|   2|
|      Engineer, land|   2|
|     Careers adviser|  15|
|Geneticist, molec...|   7|
+--------------------+----+
only showing top 5 rows



[Row(OCCUPATION='Chiropodist', Code=6),
 Row(OCCUPATION='Engineer, structural', Code=2),
 Row(OCCUPATION='Engineer, land', Code=2),
 Row(OCCUPATION='Careers adviser', Code=15),
 Row(OCCUPATION='Geneticist, molecular', Code=7)]

## Read Data Asset - Categories - See Lab Instructions
Follow the same procedure as above to insert a SparkDataFrame for Categories

In [34]:
# Insert Categories credentials in this cell after the comments
# make CERTAIN that if credentials gets generated as credentials_1 or credentials_2 or credentials_n where n is a number
# to rename as credentials
#Put cursor on the next line to Insert to code 


In [36]:
# credentials = {
#     'endpoint' : credentials['URL'],
#     'secret_key' : credentials['SECRET_KEY'], 
#     'access_key' : credentials['ACCESS_KEY'],
#     'container' : credentials['BUCKET']
# }

# import ibmos2spark

# cos = ibmos2spark.CloudObjectStorage(sc, credentials, auth_method='')

# from pyspark.sql import SparkSession
# spark = SparkSession.builder.getOrCreate()
# categories = spark.read\
#   .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
#   .option('header', 'true')\
#   .load(cos.url('data_asset/Categories*.csv', credentials['container']))
# categories.take(5)
categories = SQLContext(sc).read.csv('/project_data/data_asset/Categories_csv_5hndzh0i3z7mfgl3tzpqa1s05.csv', header='true', inferSchema = 'true')
categories.show(5)


+----+--------------------+
|Code|            Category|
+----+--------------------+
|   1|       Sports/Travel|
|   2|         Engineering|
|   3|Information Techn...|
|   4|          Journalism|
|   5|          Government|
+----+--------------------+
only showing top 5 rows



## Join the Occupation mapping file content with the Category file content. 
The "Code" column serves as the matching key. Note that we can join the two flat files as if they were database tables using the DataFrame API.  In a similar fashion we will join the resulting DataFrame with the trafficking_df Dataframe. 

In [38]:
occupations_df = occupations.join(categories,'Code','inner').drop('Code')
occupations_df.show()

+--------------------+------------+
|          OCCUPATION|    Category|
+--------------------+------------+
|         Chiropodist|     Medical|
|Engineer, structural| Engineering|
|      Engineer, land| Engineering|
|     Careers adviser|       Other|
|Geneticist, molec...|     Science|
|Electronics engineer| Engineering|
|Editor, commissio...|  Journalism|
|              Writer|  Journalism|
|Recruitment consu...|       Other|
|     Theatre manager|        Arts|
|        Statistician| Engineering|
|Merchant navy off...|       Other|
|Advertising accou...| Advertising|
|Administrator, lo...|  Government|
| Corporate treasurer|     Finance|
|           Paramedic|     Medical|
|Special effects a...|        Arts|
|           Architect|Construction|
|               Actor|        Arts|
|      Science writer|  Journalism|
+--------------------+------------+
only showing top 20 rows



In [40]:
trafficking_df = trafficking_df.join(occupations_df,'OCCUPATION','inner')
trafficking_df.show()

+--------------------+-----------+-------------+-----------+--------------------+------+-------------------+-------------+------------------+--------------------+-----------+---------------+----------------+---------------------+--------------------+-----------------------+----------------------------+--------------------+----------------------------+----------------------+------------------------------+----------------------+------------------------------+------------------------+--------------------+---+-----------+
|          OCCUPATION|INTERNAL_ID|VETTING_LEVEL|DESCRIPTION|                NAME|GENDER|         BIRTH_DATE|BIRTH_COUNTRY|BIRTH_COUNTRY_CODE|             ADDRESS|        SSN|PASSPORT_NUMBER|PASSPORT_COUNTRY|PASSPORT_COUNTRY_CODE|   COUNTRIES_VISITED|COUNTRIES_VISITED_COUNT|ARRIVAL_AIRPORT_COUNTRY_CODE|ARRIVAL_AIRPORT_IATA|ARRIVAL_AIRPORT_MUNICIPALITY|ARRIVAL_AIRPORT_REGION|DEPARTURE_AIRPORT_COUNTRY_CODE|DEPARTURE_AIRPORT_IATA|DEPARTURE_AIRPORT_MUNICIPALITY|DEPARTURE_AIRPO

<a id="transform"></a>
## Identify our labels and transform 

We will use the 'VETTING_LEVEL' column as a label for training the machine learning model.  This is where our analyst has marked the data as vetted.  

Spark ML requires that that the labels are data type Double, so we will cast the  column as Double.

withColumn() is a Spark SQL way to manipulate a dataframe.  Since an RDD is immutable, we create a new RDD each time we transform.  This code creates a new column VettingTemp and sets it to the values in "VETTING_LEVEL" cast to a Double.    It then drops column VETTING_LEVEL and renames column VettingTemp to VETTING_LEVEL.

We will also do a similar manipulation for the COUNTRIES_VISITED_COUNT

In [42]:
DataWithLabels = (trafficking_df.withColumn("VettingTemp", trafficking_df["VETTING_LEVEL"]
    .cast("Double")).drop("VETTING_LEVEL").withColumnRenamed("VettingTemp", "VETTING_LEVEL"))

DataWithLabels = (trafficking_df.withColumn("CountriesVisitedCountTemp", trafficking_df["COUNTRIES_VISITED_COUNT"]
    .cast("Double")).drop("COUNTRIES_VISITED_COUNT").withColumnRenamed("CountriesVisitedCountTemp", "COUNTRIES_VISITED_COUNT"))

We want to use year of birth intead of date of birth in our learning.  

Another way to transform an rdd in Spark is using SQL Syntax.  Here, we will be adding a new field, BIRTH_YEAR to our vetting set.  We will also just select the fields we need.

In [44]:
DataWithLabels.createOrReplaceTempView("VettingData")
AllVettingData = sqlContext.sql ("SELECT UUID, VETTING_LEVEL, NAME, Category, COUNTRIES_VISITED_COUNT, PASSPORT_COUNTRY_CODE, year(BIRTH_DATE) as BIRTH_YEAR, 1 as Counter FROM VettingData")
FilteredVettingData = AllVettingData.filter("VETTING_LEVEL==100")
FilteredVettingData.count()

816

Use pixiedust to visually explore the data.

In [ ]:
display(AllVettingData)

Now, let's look at the data we have:

VETTING_LEVEL is in four different statuses:

    10 - HIGH
    
    20 - MEDIUM
    
    30 - LOW
    
    100 - Unlabeled


Print the total number of vetting statuses 

In [48]:
print('The number of rows labeled high is {}.'.format(AllVettingData.filter(AllVettingData['VETTING_LEVEL'] == 10).count()))
print('The number of rows labeled medium is {}.'.format(AllVettingData.filter(AllVettingData['VETTING_LEVEL'] == 20).count()))
print('The number of rows labeled low is {}.'.format(AllVettingData.filter(AllVettingData['VETTING_LEVEL'] == 30).count()))
print('The number of unlabeled rows is {}.'.format(AllVettingData.filter(AllVettingData['VETTING_LEVEL'] == 100).count()))

The number of rows labeled high is 69.
The number of rows labeled medium is 95.
The number of rows labeled low is 105.
The number of unlabeled rows is 816.


The majority of the data has not been labeled (VETTING_LABEL=100 means unvetted).  We can not use it for our training data, so filter it out.
Print the total number of rows.

In [50]:
LabeledVettingData=AllVettingData.filter("VETTING_LEVEL != 100")
LabeledVettingData.count()

269

In [ ]:
display(LabeledVettingData)

<a id="engineering"></a>
## Feature Engineering.
### A feature is the elements of the data that we are using in our learning.  We need to transform each one of our features into a format that SparkML can use it.
More about the choices for feature engineering can be found here:
http://spark.apache.org/docs/2.0.0/ml-features.html#stringindexer


The first thing we will do is transform our labels (VETTING_LEVEL) into a format that we can use in the algorithm, and then get back to 'human readable' from in the end. The ML models require that the labels are in a column called 'label'.    The converter helps us transform these back in the end.



In [54]:
labelIndexer = StringIndexer(inputCol="VETTING_LEVEL", outputCol="label", handleInvalid="error")
labelModel = labelIndexer.fit(LabeledVettingData)
converter = IndexToString(inputCol="prediction", outputCol="predCategory", labels=labelModel.labels)

Next, we will process all of the features we will use. While there are a variety of choices for transforming elements, we will treat each as a String using the StringIndexer.

StringIndexer is a transformer that encodes a string column to a column of indices. The indices are ordered by value frequencies, so the most frequent value gets index 0. If the input column is numeric, it is cast to string first.

For our vetting dataset, we are interested in all string based features so we will use the StringIndexer for them.  We need to use 'handleInvalid="skip"' because not all values have been validated in our vetting set.  That means the algorithms will skip these records.

In [56]:
categoryIndexer = StringIndexer(inputCol="Category", outputCol="categoryIndex", handleInvalid="skip")
countryIndexer = StringIndexer(inputCol="PASSPORT_COUNTRY_CODE", outputCol="countryIndex", handleInvalid="skip")
yearOfBirthIndexer = StringIndexer(inputCol="BIRTH_YEAR", outputCol="birthYearIndex", handleInvalid="skip")

Now, put all of our features into a simple array using a VectorAssembler.

Note that COUNTRIIES_VISITED_COUNT is already a numeric, so we can just put that in the array as is.


In [58]:
vecAssembler = VectorAssembler(inputCols=["categoryIndex","countryIndex","birthYearIndex", "COUNTRIES_VISITED_COUNT"], outputCol="features")

Normalizer will help us normalize the features into a standard frmat.  It can help us improve the behavior of the learning algorithms.


In [60]:
normalizer = Normalizer(inputCol="features", outputCol="normFeatures", p=1.0)

<a id="model"></a>
## Declare the model that we want to use

The model here is Naive Bayes.  It will output each prediction into a 'prediction' column.  Naive Bayes  is a probabistic model that learns based on previous decisions.  We will take a best guess at the paramater 'smoothing'- SparkML will help us tune it later!


In [62]:
nb = NaiveBayes(smoothing=1.0, featuresCol="normFeatures",modelType="multinomial", labelCol="label", predictionCol="prediction")

<a id="pipeline"></a>
## Setup the Pipeline

The pipeline is the guts of the algorithm that strings all the work we've done together.

The stages are run in order and the input DataFrame is transformed as it passes through each stage.   First, comes the feature transformations, then the assembler to put them togather into one DF.  We pass that into the model. 

In machine learning, it is common to run a sequence of algorithms to process and learn from data, so this can get as complex as we want to make it!

In [64]:
pipeline = Pipeline(stages=[labelIndexer,categoryIndexer,countryIndexer, yearOfBirthIndexer, vecAssembler, normalizer, nb, converter])

<a id="train"></a>
## Train the model

We will split it into training data which is marked and test data which will be used to test the efficiency of the algorithms.

It is common to split the split up the data randomly into 70% for training and 30% for testing.  If we were to use a bigger training set, we might use an 80% / 20% split.

In [66]:
train, test = LabeledVettingData.randomSplit([70.0,30.0], seed=152345)
train.cache()
test.cache()
print('The number of records in the training data set is {}.'.format(train.count()))
print('The number of rows labeled high is {}.'.format(train.filter(train['VETTING_LEVEL'] == 10).count()))
print('The number of rows labeled medium is {}.'.format(train.filter(train['VETTING_LEVEL'] == 20).count()))
print('The number of rows labeled low is {}.'.format(train.filter(train['VETTING_LEVEL'] == 30).count()))
print('')

print('The number of records in the test data set is {}.'.format(test.count()))
print('The number of rows labeled high is {}.'.format(test.filter(test['VETTING_LEVEL'] == 10).count()))
print('The number of rows labeled medium is {}.'.format(test.filter(test['VETTING_LEVEL'] == 20).count()))
print('The number of rows labeled low is {}.'.format(test.filter(test['VETTING_LEVEL'] == 30).count()))

The number of records in the training data set is 194.
The number of rows labeled high is 54.
The number of rows labeled medium is 65.
The number of rows labeled low is 75.

The number of records in the test data set is 75.
The number of rows labeled high is 15.
The number of rows labeled medium is 30.
The number of rows labeled low is 30.


Fit the pipeline to the training data.  This will run the data through the algorithm to train it based on our labled data.

In [68]:
# Fit the pipeline to the training data assigning the result to a variable called 'model'.
model = pipeline.fit(train)

Make predictions on records in the Test data set.  This will test the model based on the 30% data we have left in reserve.  Keep in mind that the model has not seen the data in the test data set.

In [69]:
# Make predictions on the test data assigning the result to a variable called 'predictions'.
predictions = model.transform(test)

<a id="evaluate"></a>
## Show and Evaluate Results

Note that we only got a small sample of the results back because we have a very small amount of training data. 

In [70]:
predictions.count()

74

SparkML has automated ways to look at result quality called Evaluators.  More information can be found here:
http://spark.apache.org/docs/latest/mllib-evaluation-metrics.html

For simplicity here, we will use a a common evaluation method called Reciever Operator Characteristic.  This genenerally is used for binary classifiers, but we will use it because we only have 3 levels of prediction.

The curve is created by plotting the true positive rate against the false positive rate at various threshold settings. The ROC curve is thus the sensitivity as a function of fall-out. The area under the ROC curve is useful for comparing and selecting the best machine learning model for a given data set. A model with an area under the ROC curve score near 1 has very good performance. A model with a score near 0.5 is about as good as flipping a coin.

In [71]:
evaluator = BinaryClassificationEvaluator().setLabelCol("label").setMetricName("areaUnderROC")
print('Area under the ROC curve = {}.'.format(evaluator.evaluate(predictions)))

Area under the ROC curve = 0.7601532567049809.


<a id="tuning"></a>
## Automatic Algorithm Tuning - Also Called  Hyperparameter Tuning


Spark ML algorithms provide many hyperparameters for tuning models. These hyperparameters are distinct from the model parameters being optimized by Spark ML itself.  Hyperparameter tuning is accomplished by choosing the best set of parameters based on model performance on test data that the model was not trained with. All combinations of hyperparameters specified will be tried in order to find the one that leads to the model with the best evaluation result.

First we will build a paramater grid to tell SparkML what to change in its testing.  Note that we are changing all the paramaters we setup in our pipeline before - the 'smoothing' in our model, and the normalizer parameter.

In [72]:
paramGrid = (ParamGridBuilder().addGrid(nb.smoothing, [0.50,0.75,1.0])
                 .addGrid(normalizer.p, [1.0, 1.25, 1.5]).build())

Now, create a cross validator to tune the pipeline with the generated parameter grid.  Cross-validation attempts to fit the underlying estimator with user-specified combinations of parameters, cross-evaluate the fitted models, and output the best one.  

In [73]:
cv = CrossValidator().setEstimator(pipeline).setEvaluator(evaluator).setEstimatorParamMaps(paramGrid).setNumFolds(5)

Next, we will run the models through the grid we set above.  It runs a cross evaluation of the ML Pipeline to find the best model.  Note that since runs the model several times, it takes a few minutes to run.

In [74]:
cvModel = cv.fit(train)
print('Area under the ROC curve for best fitted model = {}.'.format(evaluator.evaluate(cvModel.transform(test))))

Area under the ROC curve for best fitted model = 0.7540229885057472.


Let's see what improvement we achieve by tuning the hyperparameters using cross-evaluation 

In [75]:
print('Area under the ROC curve for non-tuned model = {}.'.format(evaluator.evaluate(predictions)))
print('Area under the ROC curve for best fitted model = {}.'.format(evaluator.evaluate(cvModel.transform(test))))
print('Improvement = {0:0.2f}%'.format((evaluator.evaluate(cvModel.transform(test)) - evaluator.evaluate(predictions)) *100 / evaluator.evaluate(predictions)))

Area under the ROC curve for non-tuned model = 0.7601532567049809.
Area under the ROC curve for best fitted model = 0.7540229885057472.
Improvement = -0.81%


<a id="score"></a>
## Score the remaining records that were unscored, and load them into a Data Warehouse table. 

First, we want to only get the unvetted records.

In [76]:
NewVettingData=AllVettingData.filter("VETTING_LEVEL == 100")

Next, transform the new model with the new vetting records

In [77]:
newPreds = cvModel.transform(NewVettingData)

 Show the data we have predicted and some of the fields in the data.  

In [78]:
newPreds.select("UUID", "prediction", "predCategory", "probability", "NAME", "COUNTRIES_VISITED_COUNT", "PASSPORT_COUNTRY_CODE" ).show()

+--------------------+----------+------------+--------------------+--------------------+-----------------------+---------------------+
|                UUID|prediction|predCategory|         probability|                NAME|COUNTRIES_VISITED_COUNT|PASSPORT_COUNTRY_CODE|
+--------------------+----------+------------+--------------------+--------------------+-----------------------+---------------------+
|a7a951ca-681f-406...|       1.0|          20|[0.31164812553745...|    Cheryl Cind Lara|                    4.0|                   GH|
|81d28245-a6d4-40e...|       1.0|          20|[0.32044175065574...|     Kathleen Bailey|                    4.0|                   GH|
|eed47197-e540-4c3...|       1.0|          20|[0.35404715998879...|          Sandr Hart|                    4.0|                   GH|
|a52a6477-1f5e-4b0...|       0.0|          30|[0.51314715521269...|       Linda Stewart|                    3.0|                   GH|
|3c2c6d86-caa4-471...|       0.0|          30|[0.387848

Remember that VETTING_LEVEL is in three different statuses:


10- HIGH

20- MEDIUM

30 - LOW


Let's print the total number of vetting statuses that we predicted.  The actual predicted data is low because we only have a few vetted records.  Remember that we had to 'skip' and features that were not in our trained data, so if we didn't have someone who was born in a certain year in our training data, we won't be able to predict a result.

In [79]:
print('The number of records in the unvetted data set is {}.'.format(newPreds.count()))
print('The number of rows labeled high is {}.'.format(newPreds.filter(newPreds['predCategory'] == 10).count()))
print('The number of rows labeled medium is {}.'.format(newPreds.filter(newPreds['predCategory'] == 20).count()))
print('The number of rows labeled low is {}.'.format(newPreds.filter(newPreds['predCategory'] == 30).count()))

The number of records in the unvetted data set is 775.
The number of rows labeled high is 0.
The number of rows labeled medium is 401.
The number of rows labeled low is 374.


<a id="credentials"></a>
## Insert the database credentials - see Lab Instructions



In [80]:


# @hidden_cell
# The following code contains the credentials for a connection in your Project.
# You might want to remove those credentials before you share your notebook.
credentials = project.get_connection(name="trafficking")
print(credentials)

{'database': 'BLUDB', 'password': 'N@9pxLAaCe7jTz2WwztTP0oZH31bc', 'host': 'db2w-iwcuhlb.us-south.db2w.cloud.ibm.com', 'username': 'bluadmin'}


<a id="write"></a>
## Write the results to the DB2 Warehouse. Please replace BLB in the Table_Name with your initials.

In [81]:
valuesToWrite= newPreds.select("UUID",  "predCategory","Category")
#Please replace BLB below with your initials. 
table_Name = "FEMALE_HUMAN_TRAFFICKING_MC_ML_RESULTS"

from ingest.Connectors import Connectors

dashdbsaveoption = {
                     Connectors.DASHDB.HOST              : credentials["host"],
                     Connectors.DASHDB.DATABASE          : credentials["database"],
                     Connectors.DASHDB.USERNAME          : credentials["username"],
                     Connectors.DASHDB.PASSWORD          : credentials["password"],
                     Connectors.DASHDB.TARGET_TABLE_NAME : table_Name,
                     Connectors.DASHDB.TARGET_WRITE_MODE : 'insert' }

NewdashDBDF = valuesToWrite.write.format("com.ibm.spark.discover").options(**dashdbsaveoption).save()

## Try Random Forest Model 

In [82]:
rf = RandomForestClassifier().setLabelCol("label").setFeaturesCol("normFeatures").setNumTrees(20)
# Create new Pipeline using the RandomForest model and all the same feature transformers used above for logistic regression
pipelineRF = Pipeline(stages=[labelIndexer,categoryIndexer,countryIndexer, yearOfBirthIndexer, vecAssembler, normalizer, rf, converter])

# Train model.
modelRF = pipelineRF.fit(train)

# Make predictions.
predictionsRF = modelRF.transform(test)

evaluator = BinaryClassificationEvaluator().setLabelCol("label").setMetricName("areaUnderROC")
print('Area under the ROC curve = {}.'.format(evaluator.evaluate(predictionsRF)))

newRFPreds = modelRF.transform(NewVettingData)

print('The number of records in the unvetted data set is {}.'.format(newRFPreds.count()))
print('The number of rows labeled high is {}.'.format(newRFPreds.filter(newRFPreds['predCategory'] == 10).count()))
print('The number of rows labeled medium is {}.'.format(newRFPreds.filter(newRFPreds['predCategory'] == 20).count()))
print('The number of rows labeled low is {}.'.format(newRFPreds.filter(newRFPreds['predCategory'] == 30).count()))


Area under the ROC curve = 0.778544061302682.
The number of records in the unvetted data set is 775.
The number of rows labeled high is 118.
The number of rows labeled medium is 170.
The number of rows labeled low is 487.


## Save Model - This section below demonstrates the Watson Machine Language API which allows you to programmatically save the model that was trained to a model repository.  The model can then be deployed for use in a production application.  


### You will leverage the Machine Learning service that was created in the Lab prerequisites.  



In [87]:
import sys,os,os.path
#token = os.environ['USER_ACCESS_TOKEN']
token = "eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCJ9.eyJ1c2VybmFtZSI6Im1pY2hhZWwuY3JvbmsiLCJyb2xlIjoiVXNlciIsInBlcm1pc3Npb25zIjpbImFjY2Vzc19jYXRhbG9nIiwiYWNjZXNzX2luZm9ybWF0aW9uX2Fzc2V0cyIsInZpZXdfcXVhbGl0eSIsInZpcnR1YWxpemVfdHJhbnNmb3JtIiwibWFuYWdlX2luZm9ybWF0aW9uX2Fzc2V0cyIsImFjY2Vzc19xdWFsaXR5IiwibWFuYWdlX2Rpc2NvdmVyeSIsIm1hbmFnZV9tZXRhZGF0YV9pbXBvcnQiLCJhdXRob3JfZ292ZXJuYW5jZV9hcnRpZmFjdHMiLCJjYW5fcHJvdmlzaW9uIiwibWFuYWdlX3F1YWxpdHkiXSwic3ViIjoibWljaGFlbC5jcm9uayIsImlzcyI6IktOT1hTU08iLCJhdWQiOiJEU1giLCJ1aWQiOiIxMDAwMzMxMDE2IiwiYXV0aGVudGljYXRvciI6ImRlZmF1bHQiLCJpYXQiOjE1ODA3NTIxMDUsImV4cCI6MTU4MDc5NTMwNX0.ehEmeZrEJIXFuV4gTR-cOs82CRvqA8DEBRJEBvFo4oMBb2EtCPsdeuvAjCBSiBmr7cvPxjI5xk3dRLBt4GA2s451CGtasqW8_9bfLLMIULLeM8DAJRUKTm49Ad1wfvZN1CKtCGj1VnBcS4cO5xac_lSxOjnA_mnUTSV2_ViYJeEYYULIbZZDB4aNvvjDGLj6srz5a9XD2M5bXrYm3nnmb7t60DW0optBNF2XGGvlZfZns1dMSLMaO2V2ZF1xHtKvGYW1noqaLwD2VbeZHy60BGXIA8k9lBfptPURk1uxBvnzPAxuw6Fo3loJwNpOehewrgosWXszWAyJKlTbBjYLNQ"
print(token)
wml_credentials = {
                     "token": token,
                     "instance_id" : "wml_local",
                     "url": "https://10.75.17.115:32215",
                     "version": "2.5.0"
}
client = WatsonMachineLearningAPIClient(wml_credentials)
print(client.version)

eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCJ9.eyJ1c2VybmFtZSI6Im1pY2hhZWwuY3JvbmsiLCJyb2xlIjoiVXNlciIsInBlcm1pc3Npb25zIjpbImFjY2Vzc19jYXRhbG9nIiwiYWNjZXNzX2luZm9ybWF0aW9uX2Fzc2V0cyIsInZpZXdfcXVhbGl0eSIsInZpcnR1YWxpemVfdHJhbnNmb3JtIiwibWFuYWdlX2luZm9ybWF0aW9uX2Fzc2V0cyIsImFjY2Vzc19xdWFsaXR5IiwibWFuYWdlX2Rpc2NvdmVyeSIsIm1hbmFnZV9tZXRhZGF0YV9pbXBvcnQiLCJhdXRob3JfZ292ZXJuYW5jZV9hcnRpZmFjdHMiLCJjYW5fcHJvdmlzaW9uIiwibWFuYWdlX3F1YWxpdHkiXSwic3ViIjoibWljaGFlbC5jcm9uayIsImlzcyI6IktOT1hTU08iLCJhdWQiOiJEU1giLCJ1aWQiOiIxMDAwMzMxMDE2IiwiYXV0aGVudGljYXRvciI6ImRlZmF1bHQiLCJpYXQiOjE1ODA3NTIxMDUsImV4cCI6MTU4MDc5NTMwNX0.ehEmeZrEJIXFuV4gTR-cOs82CRvqA8DEBRJEBvFo4oMBb2EtCPsdeuvAjCBSiBmr7cvPxjI5xk3dRLBt4GA2s451CGtasqW8_9bfLLMIULLeM8DAJRUKTm49Ad1wfvZN1CKtCGj1VnBcS4cO5xac_lSxOjnA_mnUTSV2_ViYJeEYYULIbZZDB4aNvvjDGLj6srz5a9XD2M5bXrYm3nnmb7t60DW0optBNF2XGGvlZfZns1dMSLMaO2V2ZF1xHtKvGYW1noqaLwD2VbeZHy60BGXIA8k9lBfptPURk1uxBvnzPAxuw6Fo3loJwNpOehewrgosWXszWAyJKlTbBjYLNQ
1.0.60


In [88]:
client.repository.list_spaces()


------------------------------------  --------------------  ------------------------
GUID                                  NAME                  CREATED
cc1e19e2-20d8-4341-a442-2dfd5ea84377  JAIC                  2020-01-30T16:15:33.149Z
ad49bdb4-8cce-4854-af6c-c9f7a171be40  JCF Deployment Space  2020-01-29T00:43:15.716Z
------------------------------------  --------------------  ------------------------


In [89]:
# client.repository.list_models()


In [90]:
project_id = os.environ['PROJECT_ID']
print(project_id)
client.set.default_project(project_id)
#client.repository.ModelMetaNames.show()
# client.set.default_space("ad49bdb4-8cce-4854-af6c-c9f7a171be40")

98d6a078-7669-4e6c-9168-15994486dbc9


'SUCCESS'

In [91]:
model_props = {
        client.repository.ModelMetaNames.NAME: "FHT_Spark4",
        client.repository.ModelMetaNames.TYPE: "mllib_2.3",
        client.repository.ModelMetaNames.RUNTIME_UID: "spark-mllib_2.3"
}
client.set.default_space("ad49bdb4-8cce-4854-af6c-c9f7a171be40") 


Unsetting the project_id ...


'SUCCESS'

## After the cell below is executed, a model entry "FHT Spark" will appear in the Project Model section. 

In [92]:
client.repository.store_model(model=modelRF, meta_props=model_props, training_data=train, pipeline=pipelineRF)

{'metadata': {'guid': '00d8610f-9c1d-4538-bd6f-522c0c6e534c',
  'id': '00d8610f-9c1d-4538-bd6f-522c0c6e534c',
  'modified_at': '2020-02-03T17:49:11.002Z',
  'created_at': '2020-02-03T17:49:07.002Z',
  'owner': '1000331016',
  'href': '/v4/models/00d8610f-9c1d-4538-bd6f-522c0c6e534c?space_id=ad49bdb4-8cce-4854-af6c-c9f7a171be40'},
 'entity': {'name': 'FHT_Spark4',
  'training_data_references': [{'location': {'bucket': 'not_applicable'},
    'type': 'fs',
    'connection': {'access_key_id': 'not_applicable',
     'secret_access_key': 'not_applicable',
     'endpoint_url': 'not_applicable'},
    'schema': {'id': '1',
     'type': 'struct',
     'fields': [{'name': 'UUID',
       'type': 'string',
       'nullable': True,
       'metadata': {}},
      {'name': 'VETTING_LEVEL',
       'type': 'integer',
       'nullable': True,
       'metadata': {'modeling_role': 'target'}},
      {'name': 'NAME', 'type': 'string', 'nullable': True, 'metadata': {}},
      {'name': 'Category', 'type': 'stri

## Download notebook

Notebooks can be downloaded in notebook (.ipynb), Python (.py), HTML (.html), markdown (.md) or reST (.rst) format.  Use <b>File</b> > <b>Download as</b> to download the notebook in any of the formats.

<img alt="IBM Bluemix.Get started now" src="https://raw.githubusercontent.com/jpatter/LMCO/master/Lab-1/images/FileOptions.PNG" >

<a id="revert"></a>
## Revert to version 
Revert to the version you saved at the beginning of this lab.   There are two ways to do this.   First, select <b>File</b> > <b>Revert to Version</b> and choose the version you created at the beginning of the lab (versions are timestamped).
<img alt="IBM Bluemix.Get started now" src="https://raw.githubusercontent.com/jpatter/LMCO/master/Lab-1/images/FileOptions.PNG" >

The second way is to select the <b>Versions</b> icon 
<img alt="IBM Bluemix.Get started now" src="https://raw.githubusercontent.com/jpatter/LMCO/master/Lab-1/images/versions-button.png" ><br>
and then select the version you wish to revert to.   You can also delete versions from here.
<img alt="IBM Bluemix.Get started now" src="https://raw.githubusercontent.com/jpatter/LMCO/master/Lab-1/images/Versions.PNG" >

<a id="help"></a>
## Even more help

Select the <b>Find Resources in the Community</b> link to display a search bar, documentation hotlinks, and a link to Stack Overflow's Data Science Experience section.

<img alt="IBM Bluemix.Get started now" src="https://raw.githubusercontent.com/jpatter/LMCO/master/Lab-1/images/community-button.png" >

<img alt="IBM Bluemix.Get started now" src="https://raw.githubusercontent.com/jpatter/LMCO/master/Lab-1/images/Community-Resources.PNG" >